In [1]:
import numpy as np
import pandas as pd
from pysr import PySRRegressor
from matplotlib import pyplot as plt
import re
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor
#from causallearn.search.ConstraintBased.PC import pc

#import functions

In [2]:
low_scatter_para=['ETG','T-type','Bar', 'Disk', 'Ring', 'Core', 'Multiple', 'Compactness', 'AGN',
       'Pseudobulge', 'BCG', 'cD','M*_sph', 'M*_gal', 'log_B/T',
       'log_sigma0', 'log_R_e_sph_maj','log_R_e_sph_eq_kpc', 'log_n_sph_maj', 'log_n_sph_eq', 'log(I_e,sph,maj/M_Sun/pc^2)',
       'log(I_e,sph,eq/M_Sun/pc^2)', 'Concentration_Index',
       'avg_Rho_1kpc_Exact_All', 'r1_density_approx', 'log10(R10_kpc)',
       'logRho_R10_approx', 'log_rho10_Exact', 'log10(R90_kpc)',
       'logRho_R90_approx', 'log_rho_90_Exact_all', 'Rho_re_spatial',
       'SR_pc_All', 'Rho_SR_pc_All', 'CR_def1_approx_new',
       'Rho_cr_def1_approx_new', 'CR_def2_approx_new',
       'Rho_CR_def2_approx_new', 'Sr(pc)_2_using_Falserm_drho',
       'Log_Approx_Avg_density_10pc', 'log_Rho_e_Exact_new',
       'logRho_e_approx_New', 'logRho_soi_approx_new',
       'log_Rho_soi_exact_new', 'Avg_Rho_Re_Exact_all',
       'Avg_Rho_soi_exact_all', 'Avg_Rho_re_Exact_all', 'Rho_re_Exact_all',
       'Rho_r_soi_2BH_approx', 'Log_Avg_Rho_10kpc_approx',
       'Log_Avg_Rho_10kpc_exact_final', 'Log_Avg_Rho_100pc_approx',
       'Log_Avg_Rho_5kpc_approx', 'Log_Avg_rho_5kpc_exact_all', 'ube', 'bve',
       'dc', 'bvtc', 'bri25', 'mabs', 'blum', 'logblum', 'logSigma0sph',
       'LogSigma0', 'R10', 'logR10', 'logR10phi', 'Rh', 'logRh', 'logRhphi',
       'logHalo','M_BH']

In [4]:
df_full = pd.read_csv('../SMBH_Data_0911.csv',header=1)


paras=low_scatter_para.copy()
if paras[-1]!='M_BH':
    paras.append('M_BH')
paras.append('M_BH_std')

obs=df_full.copy()
obs = obs[paras].dropna(axis='index',how='any')
print(len(obs))

y = obs['M_BH'].to_numpy()
w = 1/obs['M_BH_std'].to_numpy()**2

X = obs.iloc[:,:-2].to_numpy()

100


In [22]:
# start over 100 times, each time evolve 300 generations
evolutions = 3
niterations = 3
#evolutions = 100
#niterations = 300


denoise=False
ncyclesperiteration=5000
adaptive_parsimony_scaling=20
verbosity=0
return_model=True
maxsize=20
optimizer_algorithm="NelderMead"
optimizer_iterations=100
should_simplify=True
temp_equation_file=True  # turn this on will prevent the auto-save of hall_of_fame
tempdir='/data/zj448/SR/Ultimate_paper/temp/'  # dummy dir to save temp files and then being auto deleted

In [34]:
for epoch in range(evolutions):

    warm_start=False

    for iter in range(niterations):
        model = PySRRegressor(
            binary_operators=["+", "-", "*", "/","pow"],
            unary_operators=["exp","log10"],
            constraints={"pow": (9, 1)}, # power laws can have 9 complexity left argument, but only 1 complexity in the right argument.
            warm_start=warm_start,
            denoise=denoise,
            niterations=1,
            ncyclesperiteration=ncyclesperiteration,
            adaptive_parsimony_scaling=adaptive_parsimony_scaling,
            verbosity=verbosity,
            precision=64,
            maxsize=maxsize,
            optimizer_algorithm=optimizer_algorithm,
            optimizer_iterations=optimizer_iterations,
            should_simplify=should_simplify,
            temp_equation_file=temp_equation_file,
            tempdir=tempdir,
            )
        

        model.fit(X=X, y=y, weights=w)


        equations = model.equations_
    
        number_matching_pattern = r"(?<![a-zA-Z0-9_.])[+-]?(\d+\.\d+|\.\d+|\d+\.|\d+)(?:[eE][-+]?\d+)?"

        variable_matching_pattern = r'\bx\d+'


        # Count number of constants:
        equations["number_constants"] = [len(re.findall(number_matching_pattern, eq)) for eq in equations["equation"]]

        # count number of (unique) variables
        equations["number_variables"] = [len(re.findall(variable_matching_pattern, eq)) for eq in equations["equation"]]
        equations["unique_number_variables"] = [len(set(re.findall(variable_matching_pattern, eq))) for eq in equations["equation"]]

        # Compute log likelihood (for example)
        equations["log_like"] = - equations["loss"] * len(X)

        # Compute AIC:
        equations["aic"] = 2 * equations["number_constants"] - 2 * equations["log_like"]

        equations.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/pareto_'+str(epoch)+'_'+str(iter)+'.csv')

        warm_start=True

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1937: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(
/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/zj448/minicon

In [35]:
equations

,complexity,loss,score,equation,sympy_format,lambda_format,number_constants,number_variables,unique_number_variables,log_like,aic
0,1,5.284935,0.000000,x69,x69,PySRFunction(X=>x69),0,1,1,-528.493489,1056.986978
1,3,0.312159,1.414552,(x13 + -2.7246199129259274),x13 - 2.7246199129259274,PySRFunction(X=>x13 - 2.7246199129259274),1,1,1,-31.215879,64.431757
2,4,0.280564,0.106711,(x69 - exp(x3)),x69 - exp(x3),PySRFunction(X=>x69 - exp(x3)),0,2,2,-28.056389,56.112778
3,5,0.189309,0.393419,(x13 + (x16 - 3.1255318803272307)),x13 + x16 - 3.1255318803272307,PySRFunction(X=>x13 + x16 - 3.1255318803272307),1,2,2,-18.930941,39.861882
4,6,0.143736,0.275406,(exp(exp(exp(x53))) - -4.332799083256326),exp(exp(exp(x53))) + 4.332799083256326,PySRFunction(X=>exp(exp(exp(x53))) + 4.3327990...,1,1,1,-14.373573,30.747146
5,8,0.142545,0.004160,((x69 - exp(1.0451524346431438 ^ x30)) + x17),x17 + x69 - exp(1.0451524346431438**x30),PySRFunction(X=>x17 + x69 - exp(1.045152434643...,1,3,3,-14.254474,30.508948
6,9,0.140679,0.013176,((((x69 + x16) / 1.2782770440195763) - 0.44736...,0.78230302631069185*x16 + x5 + 0.7823030263106...,PySRFunction(X=>0.78230302631069185*x16 + x5 +...,2,3,3,-14.067884,32.135768
7,11,0.129010,0.043297,((((x69 + x16) / 1.2782770440195763) - 0.44736...,0.78230302631069185*x16 + 0.80306845625943204*...,PySRFunction(X=>0.78230302631069185*x16 + 0.80...,3,3,3,-12.900953,31.801906
8,13,0.058537,0.395110,(x13 + (-0.4083174827029943 - log10(exp(((x42 ...,x13 - log(exp(x5 + 6.715533487955264*x42/x39))...,PySRFunction(X=>x13 - log(exp(x5 + 6.715533487...,2,4,4,-5.853746,15.707491


In [36]:
equations = pd.read_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/pareto_0_1.csv',index_col=0)
equations

,complexity,loss,score,equation,sympy_format,lambda_format,number_constants,number_variables,unique_number_variables,log_like,aic
0,1,5.284935,0.000000,x69,x69,PySRFunction(X=>x69),0,1,1,-528.493489,1056.986978
1,3,0.312159,1.414552,(x13 + -2.7246199129259274),x13 - 2.7246199129259274,PySRFunction(X=>x13 - 2.7246199129259274),1,1,1,-31.215879,64.431757
2,4,0.280564,0.106711,(x69 - exp(x3)),x69 - exp(x3),PySRFunction(X=>x69 - exp(x3)),0,2,2,-28.056389,56.112778
3,5,0.189309,0.393419,(x13 + (x16 - 3.1255318803272307)),x13 + x16 - 3.1255318803272307,PySRFunction(X=>x13 + x16 - 3.1255318803272307),1,2,2,-18.930941,39.861882
4,6,0.143736,0.275406,(exp(exp(exp(x53))) - -4.332799083256326),exp(exp(exp(x53))) + 4.332799083256326,PySRFunction(X=>exp(exp(exp(x53))) + 4.3327990...,1,1,1,-14.373573,30.747146
5,8,0.142545,0.004160,((x69 - exp(1.0451524346431438 ^ x30)) + x17),x17 + x69 - exp(1.0451524346431438**x30),PySRFunction(X=>x17 + x69 - exp(1.045152434643...,1,3,3,-14.254474,30.508948
6,9,0.140679,0.013176,((((x69 + x16) / 1.2782770440195763) - 0.44736...,0.78230302631069185*x16 + x5 + 0.7823030263106...,PySRFunction(X=>0.78230302631069185*x16 + x5 +...,2,3,3,-14.067884,32.135768
7,11,0.129010,0.043297,((((x69 + x16) / 1.2782770440195763) - 0.44736...,0.78230302631069185*x16 + 0.80306845625943204*...,PySRFunction(X=>0.78230302631069185*x16 + 0.80...,3,3,3,-12.900953,31.801906
8,13,0.058537,0.395110,(x13 + (-0.4083174827029943 - log10(exp(((x42 ...,x13 - log(exp(x5 + 6.715533487955264*x42/x39))...,PySRFunction(X=>x13 - log(exp(x5 + 6.715533487...,2,4,4,-5.853746,15.707491
